In [43]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import merge, Convolution2D, MaxPooling2D, Input, add
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU


def makeModel(nbChannels, shape1, shape2, nbClasses, nbRCL=5, nbFilters=128, filtersize = 3):


	model = BuildRCNN(nbChannels, shape1, shape2, nbClasses, nbRCL, nbFilters, filtersize)
	return model

def BuildRCNN(nbChannels, shape1, shape2, nbClasses, nbRCL, nbFilters, filtersize):
    
    def RCL_block(l_settings, l, pool=True, increase_dim=False):
        input_num_filters = l_settings.output_shape[1]
        if increase_dim:
            out_num_filters = input_num_filters*2
        else:
            out_num_filters = input_num_filters
		   
        conv1 = Convolution2D(out_num_filters, 1, 1, border_mode='same')
        stack1 = conv1(l)   	
        stack2 = BatchNormalization()(stack1)
        stack3 = PReLU()(stack2)
        
        conv2 = Convolution2D(out_num_filters, filtersize, filtersize, border_mode='same', init = 'he_normal')
        stack4 = conv2(stack3)
        stack5 = add([stack1, stack4])
        stack6 = BatchNormalization()(stack5)
        stack7 = PReLU()(stack6)
    	
        conv3 = Convolution2D(out_num_filters, filtersize, filtersize, border_mode='same', weights = conv2.get_weights())
        stack8 = conv3(stack7)
        stack9 = add([stack1, stack8])
        stack10 = BatchNormalization()(stack9)
        stack11 = PReLU()(stack10)    
        
        conv4 = Convolution2D(out_num_filters, filtersize, filtersize, border_mode='same', weights = conv2.get_weights())
        stack12 = conv4(stack11)
        stack13 = add([stack1, stack12])
        stack14 = BatchNormalization()(stack13)
        stack15 = PReLU()(stack14)    
        
        if pool:
            stack16 = MaxPooling2D((2, 2), border_mode='same')(stack15) 
            stack17 = Dropout(0.1)(stack16)
        else:
            stack17 = Dropout(0.1)(stack15)
            
        return stack17

    #Build Network
    input_img = Input(shape=(nbChannels, shape1, shape2))
    conv_l = Convolution2D(nbFilters, filtersize, filtersize, border_mode='same', activation='relu')
    l = conv_l(input_img)
    
    for n in range(nbRCL):
        if n % 2 ==0:
            l = RCL_block(conv_l, l, pool=False)
        else:
            l = RCL_block(conv_l, l, pool=True)
    
    out = Flatten()(l)        
    l_out = Dense(nbClasses, activation = 'softmax')(out)
    
    model = Model(input = input_img, output = l_out)
    
    return model


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [38]:
print('train :', len(mnist.train.images))
print('test :', len(mnist.test.images))


train : 55000
test : 10000


In [39]:
train_X = mnist.train.images
test_X = mnist.test.images

train_y = mnist.train.labels
test_y = mnist.test.labels

In [40]:
train_X_re = train_X.reshape(55000, 1, 28, 28)
test_X_re = test_X.reshape(10000, 1, 28, 28)

In [41]:
train_X_re.shape, test_X_re.shape

((55000, 1, 28, 28), (10000, 1, 28, 28))

In [26]:
train_y.shape

(55000, 10)

In [25]:
makeModel

<function __main__.makeModel(nbChannels, shape1, shape2, nbClasses, nbRCL=5, nbFilters=128, filtersize=3)>

In [34]:
model = makeModel(1, train_X_re.shape[1], train_X_re.shape[2], train_y.shape[1] )

/home/user4/.conda/envs/tensor/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", padding="same")`
/home/user4/.conda/envs/tensor/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (1, 1), padding="same")`
/home/user4/.conda/envs/tensor/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (3, 3), padding="same", kernel_initializer="he_normal")`
/home/user4/.conda/envs/tensor/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (3, 3), weights=[array([[[..., padding="same")`
/home/user4/.conda/envs/tensor/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (3, 3), weights=[array([[[..., padding="same")`
/

In [36]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [44]:
model.fit(train_X_re, train_y, epochs = 20, batch_size = 100)

Epoch 1/20
55000/55000 [==============================] - 21s 388us/step - loss: 0.7175
Epoch 2/20
55000/55000 [==============================] - 21s 379us/step - loss: 0.7015
Epoch 3/20
55000/55000 [==============================] - 21s 386us/step - loss: 0.6769
Epoch 4/20
55000/55000 [==============================] - 21s 383us/step - loss: 0.6609
Epoch 5/20
55000/55000 [==============================] - 21s 375us/step - loss: 0.6391
Epoch 6/20
55000/55000 [==============================] - 21s 384us/step - loss: 0.6346
Epoch 7/20
55000/55000 [==============================] - 21s 383us/step - loss: 0.6156
Epoch 8/20
55000/55000 [==============================] - 21s 385us/step - loss: 0.6121
Epoch 9/20
55000/55000 [==============================] - 21s 385us/step - loss: 0.5982
Epoch 10/20
55000/55000 [==============================] - 21s 385us/step - loss: 0.5838
Epoch 11/20
55000/55000 [==============================] - 21s 382us/step - loss: 0.5754
Epoch 12/20
55000/55000 [=====

In [55]:
import numpy as np

pred = np.argmax(model.predict(test_X_re), axis=1)

np.mean(pred == np.argmax(test_y, axis=1))

0.9303